In [75]:
import json
import pandas as pd
import os

In [76]:
path = 'C:/Users/USER/Desktop/data2/Validation/[라벨]General_val_A/district/'

In [77]:
keypoints = []
get_on = []
get_off = []

for i in os.listdir(path):
    # os.listdir(path+i)
    file_list = os.listdir(path+i)
       
    for a in file_list:
        with open ((path+i+'/'+a),"r") as j:
            contents = json.loads(j.read())
            if len(contents['annotations']) != 0:
                keypoints.append(contents['annotations'][0]['keypoints'])
                get_on.append(contents['annotations'][0]['get_on'])
                get_off.append(contents['annotations'][0]['get_off'])        

In [78]:
# file_list = os.listdir(path)
# file_list_py = [file for file in file_list if file.endswith('.json')] ## 파일명 끝이 .json인 경우

In [79]:
df = pd.DataFrame({'keypoints':keypoints,
                   'get_on':get_on,
                   'get_off':get_off})

In [80]:
get = []
for i in range(len(get_on)):
    if (df['get_on'][i] == False) and (df['get_off'][i] == False):
        get.append(2)
    elif (df['get_on'][i] == True) and (df['get_off'][i] == False):
        get.append(1)
    else:
        get.append(0)

In [81]:
df['get'] = get

In [82]:
df

,keypoints,get_on,get_off,get
0,"[279, 703, 1, 337, 604, 2, 313, 413, 2, 314, 3...",False,True,0
1,"[348, 700, 2, 496, 705, 2, 469, 504, 2, 507, 4...",False,True,0
2,"[748, 880, 2, 695, 729, 2, 618, 529, 2, 692, 4...",False,True,0
3,"[800, 898, 2, 849, 780, 2, 852, 565, 2, 908, 5...",False,True,0
4,"[833, 892, 2, 989, 820, 2, 1044, 609, 2, 1107,...",False,True,0
...,...,...,...,...
16488,"[1058, 189, 2, 1059, 154, 2, 1056, 115, 2, 103...",False,False,2
16489,"[992, 168, 1, 997, 138, 1, 1000, 110, 2, 986, ...",False,False,2
16490,"[943, 256, 1, 935, 225, 1, 927, 185, 2, 934, 1...",False,False,2
16491,"[1668, 453, 2, 1663, 399, 2, 1683, 350, 2, 171...",False,False,2


In [83]:
df['get_off']

0         True
1         True
2         True
3         True
4         True
         ...  
16488    False
16489    False
16490    False
16491    False
16492    False
Name: get_off, Length: 16493, dtype: bool

In [84]:
import numpy as np

In [86]:
keypoints = np.array(keypoints)

In [87]:
keypoints = keypoints.reshape(-1,16,1,3)

In [88]:
keypoints.shape

(16493, 16, 1, 3)

In [89]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(get, num_classes=3)

In [90]:
y.shape

(16493, 3)

In [91]:
from sklearn.model_selection import train_test_split

In [92]:
x_train,x_test,y_train,y_test=train_test_split(keypoints,y,test_size=0.2,random_state=42)

In [93]:
import tensorflow as tf

In [94]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,RMSprop,Adagrad
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPool2D

In [95]:
model=Sequential()
model.add(Conv2D(30, kernel_size=3, padding='same', input_shape=(16, 1, 3))) 
# 이미지 특징 다운샘플링 Layer => MaxPool2D
# 1차원으로 데이터 형태 변환 Layer => Flatten
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax')) # 1차원

In [96]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 16, 1, 30)         840       
                                                                 
 flatten_3 (Flatten)         (None, 480)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                30784     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 3)                 195       
                                                                 
Total params: 31,819
Trainable params: 31,819
Non-trainable params: 0
_________________________________________________________________


In [97]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [98]:
model.fit(x_train,y_train,epochs=30,batch_size=64,validation_split=0.2)

Epoch 1/30
165/165 [==============================] - 3s 8ms/step - loss: 11.7300 - accuracy: 0.5028 - val_loss: 0.8292 - val_accuracy: 0.5699
Epoch 2/30
165/165 [==============================] - 1s 4ms/step - loss: 0.8816 - accuracy: 0.5628 - val_loss: 0.7939 - val_accuracy: 0.5623
Epoch 3/30
165/165 [==============================] - 1s 5ms/step - loss: 0.8612 - accuracy: 0.5709 - val_loss: 0.8354 - val_accuracy: 0.5710
Epoch 4/30
165/165 [==============================] - 1s 5ms/step - loss: 0.8304 - accuracy: 0.5702 - val_loss: 0.7642 - val_accuracy: 0.5809
Epoch 5/30
165/165 [==============================] - 1s 5ms/step - loss: 0.8269 - accuracy: 0.5719 - val_loss: 0.7477 - val_accuracy: 0.5854
Epoch 6/30
165/165 [==============================] - 1s 5ms/step - loss: 0.8239 - accuracy: 0.5710 - val_loss: 0.7736 - val_accuracy: 0.5654
Epoch 7/30
165/165 [==============================] - 1s 5ms/step - loss: 0.8239 - accuracy: 0.5738 - val_loss: 0.7688 - val_accuracy: 0.5972
Epoch

In [99]:
model.evaluate(x_test,y_test)

104/104 [==============================] - 0s 2ms/step - loss: 0.6684 - accuracy: 0.7142


[0.6683507561683655, 0.7141557931900024]